In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import datetime
import seaborn as sns

#import pydicom
import time
from functools import partial
import gc
import operator 
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F
from sklearn.model_selection import KFold
from tqdm import tqdm, tqdm_notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
%load_ext autoreload
%autoreload 2
%matplotlib inline
from skimage.io import imread,imshow
from helper import *
import helper
import torchvision.models as models
from torch.optim import Adam
from defenitions import *

/home/reina/anaconda3/envs/RSNA/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/reina/anaconda3/envs/RSNA/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [2]:
SEED = 432
device=device_by_name("Tesla")
#device=device_by_name("RTX")
#device=device_by_name("1060")
torch.cuda.set_device(device)
#device = "cpu"
sendmeemail=Email_Progress(my_gmail,my_pass,to_email,'Densenet161-Copy2-2 results')

In [3]:
def get_submission(test_df,pred):
    epidural_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_epidural','Label':torch.sigmoid(pred[:,0])})
    intraparenchymal_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_intraparenchymal','Label':torch.sigmoid(pred[:,1])})
    intraventricular_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_intraventricular','Label':torch.sigmoid(pred[:,2])})
    subarachnoid_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_subarachnoid','Label':torch.sigmoid(pred[:,3])})
    subdural_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_subdural','Label':torch.sigmoid(pred[:,4])})
    any_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_any','Label':torch.sigmoid(pred[:,5])}) 
    return pd.concat([epidural_df,
                        intraparenchymal_df,
                        intraventricular_df,
                        subarachnoid_df,
                        subdural_df,
                        any_df]).sort_values('ID').reset_index(drop=True)

In [4]:
def get_submission_ids(image_ids,pred,do_sigmoid=True):
    if do_sigmoid:
        func = lambda x:torch.sigmoid(x)
    else:
        func = lambda x:x
    epidural_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_epidural','Label':func(pred[:,0])})
    intraparenchymal_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_intraparenchymal','Label':func(pred[:,1])})
    intraventricular_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_intraventricular','Label':func(pred[:,2])})
    subarachnoid_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_subarachnoid','Label':func(pred[:,3])})
    subdural_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_subdural','Label':func(pred[:,4])})
    any_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_any','Label':func(pred[:,5])}) 
    return pd.concat([epidural_df,
                        intraparenchymal_df,
                        intraventricular_df,
                        subarachnoid_df,
                        subdural_df,
                        any_df]).sort_values('ID').reset_index(drop=True)

In [5]:
old_train_df = pd.read_csv(data_dir+'train.csv')
old_train_df.shape
old_train_df=old_train_df[~old_train_df.PatientID.isin(bad_images)].reset_index(drop=True)
old_train_df=old_train_df.drop_duplicates().reset_index(drop=True)
old_train_df.shape
old_train_df.head()

train_df = pd.read_csv(data_dir+'train_stage2.csv')
train_df.shape


old_test_df = pd.read_csv(data_dir+'test.csv')
old_test_df.head()
assert old_test_df.shape[0]+old_train_df.shape[0]==train_df.shape[0]

n_splits=5
old_split_sid = old_train_df.PID.unique()
old_split_sid.shape[0]
old_splits=list(KFold(n_splits=n_splits,shuffle=True, random_state=SEED).split(old_split_sid))


new_split_sid=np.array(list(set(train_df.PID.unique()).difference(old_train_df.PID.unique())))
new_split_sid.shape[0]
new_splits=list(KFold(n_splits=n_splits,shuffle=True, random_state=SEED).split(new_split_sid))
split_sid=np.concatenate([old_split_sid,new_split_sid])
split_sid.shape[0]

splits=[]
for i in range(len(old_splits)):
    splits.append((np.concatenate([old_splits[i][0],new_splits[i][0]+old_split_sid.shape[0]]),
                  np.concatenate([old_splits[i][1],new_splits[i][1]+old_split_sid.shape[0]])))

for i in range(n_splits):
    for j in range(n_splits):
        idx_train = train_df[train_df.PID.isin(set(split_sid[splits[i][0]]))].index.values
        idx_validate =  train_df[train_df.PID.isin(set(split_sid[splits[j][1]]))].index.values
        if i==j:
            assert np.unique(np.concatenate([idx_train,idx_validate])).shape[0]==idx_train.shape[0]+idx_validate.shape[0]
        else:
            assert np.unique(np.concatenate([idx_train,idx_validate])).shape[0]==idx_train.shape[0]

(674252, 15)

(674252, 15)

,PatientID,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any,PID,StudyI,SeriesI,WindowCenter,WindowWidth,ImagePositionZ,ImagePositionX,ImagePositionY
0,63eb1e259,0,0,0,0,0,0,a449357f,62d125e5b2,0be5c0d1b3,"['00036', '00036']","['00080', '00080']",180.199951,-125.0,-8.000000
1,2669954a7,0,0,0,0,0,0,363d5865,a20b80c7bf,3564d584db,"['00047', '00047']","['00080', '00080']",922.530821,-156.0,45.572849
2,52c9913b1,0,0,0,0,0,0,9c2b4bd7,3e3634f8cf,973274ffc9,40,150,4.455000,-125.0,-115.063000
3,4e6ff6126,0,0,0,0,0,0,3ae81c2d,a1390c15c2,e5ccad8244,"['00036', '00036']","['00080', '00080']",100.000000,-99.5,28.500000
4,7858edd88,0,0,0,0,0,0,c1867feb,c73e81ed3a,28e0531b3a,40,100,145.793000,-125.0,-132.190000


(752797, 15)

,PatientID,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any,SeriesI,PID,StudyI,WindowCenter,WindowWidth,ImagePositionZ,ImagePositionX,ImagePositionY
0,28fbab7eb,0.5,0.5,0.5,0.5,0.5,0.5,ebfd7e4506,cf1b6b11,93407cadbb,30,80,158.458000,-125.0,-135.598000
1,877923b8b,0.5,0.5,0.5,0.5,0.5,0.5,6d95084e15,ad8ea58f,a337baa067,30,80,138.729050,-125.0,-101.797981
2,a591477cb,0.5,0.5,0.5,0.5,0.5,0.5,8e06b2c9e0,ecfb278b,0cfe838d54,30,80,60.830002,-125.0,-133.300003
3,42217c898,0.5,0.5,0.5,0.5,0.5,0.5,e800f419cf,e96e31f4,c497ac5bad,30,80,55.388000,-125.0,-146.081000
4,a130c4d2f,0.5,0.5,0.5,0.5,0.5,0.5,faeb7454f3,69affa42,854e4fbc01,30,80,33.516888,-125.0,-118.689819


17079

1859

18938

In [6]:
pickle_file=open(outputs_dir+'PID_splits_5_stage_2.pkl'.format(n_splits),'rb')
split_sid,splits=pickle.load(pickle_file)
pickle_file.close()


In [7]:
def my_loss(y_pred,y_true,weights):
    window=(y_true>=0).to(torch.float)
    loss = (F.binary_cross_entropy_with_logits(y_pred,y_true,reduction='none')*window*weights.expand_as(y_true)).mean()/(window.mean()+1e-7)
    return loss

In [8]:
class Metric():
    def __init__(self,weights,k=0.03):
        self.weights=weights
        self.k=k
        self.zero()
        
    def zero(self):
        self.loss_sum=0.
        self.loss_count=0.
        self.lossf=0.
        
    def calc(self,y_pred,y_true,prefix=""):
        window=(y_true>=0).to(torch.float)
        loss = (F.binary_cross_entropy_with_logits(y_pred,y_true,reduction='none')*window*self.weights.expand_as(y_true)).mean()/(window.mean()+1e-5)
        self.lossf=self.lossf*(1-self.k)+loss*self.k
        self.loss_sum=self.loss_sum+loss*window.sum()
        self.loss_count=self.loss_count+window.sum()
        return({prefix+'mloss':self.lossf})    
        
    def calc_sums(self,prefix=""):
        return({prefix+'mloss_tot':self.loss_sum/self.loss_count})    



In [9]:
#features=(features-features.mean())/features.std()

In [10]:
class SimpleModel(nn.Module):
    def __init__(self,in_size):
        super(SimpleModel, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 128, (7,in_size), padding=(3,0))
        self.bn0=torch.nn.BatchNorm1d(128)
        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(128, 128, 5, padding=2)
        self.bn1=torch.nn.BatchNorm1d(128)
        self.relu1=torch.nn.ReLU()
        self.conv1d2=torch.nn.Conv1d(128, 64, 3, padding=1)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=torch.nn.ReLU()
        self.conv1d3=torch.nn.Conv1d(64, 6, 3, padding=1)
        
        
    def forward(self, x):
        x = self.conv2d1(x.unsqueeze(1)).squeeze(-1)
        x = self.bn0(x)
        x = self.relu0(x)
        x = self.conv1d1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.conv1d2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        out = self.conv1d3(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [11]:
class SimpleModel2(nn.Module):
    def __init__(self,in_size):
        super(SimpleModel2, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 128, (9,in_size), padding=(4,0))
        self.bn0=torch.nn.BatchNorm1d(128)
        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(128, 128, 7, padding=3)
        self.bn1=torch.nn.BatchNorm1d(128)
        self.relu1=torch.nn.ReLU()
        self.conv1d2=torch.nn.Conv1d(128, 64, 5, padding=2)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=torch.nn.ReLU()
        self.conv1d3=torch.nn.Conv1d(64, 6, 3, padding=1)
        
        
    def forward(self, x):
        x = self.conv2d1(x.unsqueeze(1)).squeeze(-1)
        x = self.bn0(x)
        x = self.relu0(x)
        x = self.conv1d1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.conv1d2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        out = self.conv1d3(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [12]:
class ClassModel(nn.Module):
    def __init__(self,in_size):
        super(ClassModel, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 128, (9,in_size), padding=(4,0))
        self.bn0=torch.nn.BatchNorm1d(128)
        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(128, 128, 7, padding=3)
        self.bn1=torch.nn.BatchNorm1d(128)
        self.relu1=torch.nn.ReLU()
        self.conv1d2=torch.nn.Conv1d(128, 64, 5, padding=2)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=torch.nn.ReLU()
        self.conv1d3=torch.nn.Conv1d(128, 6, 3, padding=1)
        
        self.conv2d1class=torch.nn.Conv2d(1, 128, (9,in_size), padding=(4,0))
        self.bn0class=torch.nn.BatchNorm1d(128)
        self.maxpool1class=torch.nn.MaxPool1d(3)
        self.conv1d1class=torch.nn.Conv1d(128, 128, 3, padding=1)
        self.bn1class=torch.nn.BatchNorm1d(128)
        self.maxpool2class=torch.nn.MaxPool1d(3)
        self.conv1d2class=torch.nn.Conv1d(128, 64, 2, padding=1)
        self.bn2class=torch.nn.BatchNorm1d(64)

        
        
    def forward(self, x):
        z=x
        x = self.conv2d1(x.unsqueeze(1)).squeeze(-1)
        x = self.bn0(x)
        x = self.relu0(x)
        x = self.conv1d1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.conv1d2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        z=self.conv2d1class(z.unsqueeze(1)).squeeze(-1)
        z=self.bn0class(z)
        z=self.maxpool1class(z)
        z=self.conv1d1class(z)
        z=self.maxpool2class(z)
        z=self.conv1d2class(z)
        z=self.bn2class(z)
        z=F.max_pool1d(z,kernel_size=z.shape[-1])
        z=z.expand_as(x)
        x=torch.cat([x,z],1)
        out = self.conv1d3(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [13]:
class ResModel(nn.Module):
    def __init__(self,in_size):
        super(ResModel, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 64, (9,in_size), padding=(4,0))
        self.bn0=torch.nn.BatchNorm1d(64)
        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(64, 64, 7, padding=3)
        self.bn1=torch.nn.BatchNorm1d(64)
        self.relu1=torch.nn.ReLU()
        self.conv1d2=torch.nn.Conv1d(128, 64, 5, padding=2)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=torch.nn.ReLU()
        self.conv1d3=torch.nn.Conv1d(192, 6, 3, padding=1)
        
        
    def forward(self, x):
        x=x.unsqueeze(1)
        x = self.conv2d1(x).squeeze(-1)
        x = self.bn0(x)
        x0 = self.relu0(x)
        x = self.conv1d1(x0)
        x = self.bn1(x)
        x1 = self.relu1(x)
        x = torch.cat([x0,x1],1)
        x = self.conv1d2(x)
        x = self.bn2(x)
        x2 = self.relu2(x)
        x = torch.cat([x0,x1,x2],1)
        out = self.conv1d3(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [14]:
class ResModelPool(nn.Module):
    def __init__(self,in_size):
        super(ResModelPool, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 64, (9,in_size),stride=(1,in_size), padding=(4,0))
        self.bn0=torch.nn.BatchNorm1d(64)
#        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(64, 64, 7, padding=3)
        self.bn1=torch.nn.BatchNorm1d(64)
        self.relu1=torch.nn.ReLU()
        self.conv1d2=torch.nn.Conv1d(128, 64, 5, padding=2)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=torch.nn.ReLU()
        self.conv1d3=torch.nn.Conv1d(192, 6, 3, padding=1)
        
        
    def forward(self, x):
        x=x.unsqueeze(1)
        x = self.conv2d1(x)
        x=F.max_pool2d(x,kernel_size=(1,x.shape[-1])).squeeze(-1)        
        x0 = self.bn0(x)
#        x0 = self.relu0(x)
        x = self.conv1d1(x0)
        x = self.bn1(x)
        x1 = self.relu1(x)
        x = torch.cat([x0,x1],1)
        x = self.conv1d2(x)
        x = self.bn2(x)
        x2 = self.relu2(x)
        x = torch.cat([x0,x1,x2],1)
        out = self.conv1d3(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [15]:
class ResDropModel(nn.Module):
    def __init__(self,in_size,dropout=0.2):
        super(ResDropModel, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 64, (9,in_size), padding=(4,0))
        self.bn0=torch.nn.BatchNorm1d(64)
        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(64, 64, 7, padding=3)
        self.bn1=torch.nn.BatchNorm1d(64)
        self.relu1=torch.nn.ReLU()
        self.conv1d2=torch.nn.Conv1d(128, 64, 5, padding=2)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=torch.nn.ReLU()
        self.conv1d3=torch.nn.Conv1d(192, 6, 3, padding=1)
        self.dropout=dropout
        
    def forward(self, x):
        x = self.conv2d1(x.unsqueeze(1)).squeeze(-1)
        x = self.bn0(x)
        x = F.dropout(x,self.dropout)
        x0 = self.relu0(x) 
        x = self.conv1d1(x0)
#        x = self.bn1(x)
        x = F.dropout(x,self.dropout)
        x1 = self.relu1(x)
        x = torch.cat([x0,x1],1)
        x = self.conv1d2(x)
#        x = self.bn2(x)
        x = F.dropout(x,self.dropout)
        x2 = self.relu2(x)
        x = torch.cat([x0,x1,x2],1)
        x = F.dropout(x,self.dropout)
        out = self.conv1d3(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [16]:
fn = partial(torch.clamp,min=0,max=1)

In [17]:
class GenReLU(nn.Module):
    def __init__(self,leak=0,add=0,clamp=None):
        super(GenReLU, self).__init__()
        self.leak,self.add=leak,add
        if isinstance(clamp,tuple):
            self.clamp = partial(torch.clamp,min=clamp[0],max=clamp[1])
        elif clamp:
            self.clamp = partial(torch.clamp,min=-clamp,max=clamp)
        else:
            self.clamp=None
            
        
    def forward(self,x):
        x = F.leaky_relu(x,self.leak)
        if self.add:
            x=x+self.add
        if self.clamp:
            x = self.clamp(x)
        return x
        
        

In [18]:

class ResModelIn(nn.Module):
    def __init__(self,in_size):
        super(ResModelIn, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 64, (9,in_size), padding=(4,0))
        self.bn0=torch.nn.BatchNorm1d(64)
        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(64, 64, 3, padding=1)
        self.bn1=torch.nn.BatchNorm1d(64)
        self.relu1=torch.nn.ReLU()
        self.conv1d2=torch.nn.Conv1d(128, 64, 3, padding=1)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=torch.nn.ReLU()
        self.conv1d3=torch.nn.Conv1d(192, 6, 3, padding=1)

        
        
    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.conv2d1(x).squeeze(-1)
        x = self.bn0(x)
        x0 = self.relu0(x)
        x = self.conv1d1(x0)
        x = self.bn1(x)
        x1 = self.relu1(x)
        x = torch.cat([x0,x1],1)
        x = self.conv1d2(x)
        x = self.bn2(x)
        x2 = self.relu2(x)
        x = torch.cat([x0,x1,x2],1)
        x = self.conv1d3(x)
        out = x.transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [19]:

class ResModelInR(nn.Module):
    def __init__(self,in_size):
        super(ResModelInR, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 64, (9,in_size), padding=(4,0))
        self.bn0=torch.nn.BatchNorm1d(64)
        self.relu0= GenReLU(leak=0.01,add=-0.3,clamp=(-1,3))
        self.conv1d1=torch.nn.Conv1d(64, 64, 3, padding=1)
        self.bn1=torch.nn.BatchNorm1d(64)
        self.relu1=GenReLU(leak=0.01,add=-0.3,clamp=(-1,3))
        self.conv1d2=torch.nn.Conv1d(128, 64, 3, padding=1)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=GenReLU(leak=0,add=0,clamp=(0,2))
        self.conv1d3=torch.nn.Conv1d(192, 6, 3, padding=1)

        
        
    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.conv2d1(x).squeeze(-1)
        x = self.bn0(x)
        x0 = self.relu0(x)
        x = self.conv1d1(x0)
        x = self.bn1(x)
        x1 = self.relu1(x)
        x = torch.cat([x0,x1],1)
        x = self.conv1d2(x)
        x = self.bn2(x)
        x2 = self.relu2(x)
        x = torch.cat([x0,x1,x2],1)
        x = self.conv1d3(x)
        out = x.transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [20]:
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 128, (5,2208), padding=(2,0))
        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(128, 6, 1)

        
        
    def forward(self, x):
        x = self.conv2d1(x.unsqueeze(1)).squeeze(-1)
        x = self.relu0(x)
        out = self.conv1d1(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [21]:
%matplotlib nbagg

for num_split in range(5):
    multi=3
    model_name,version = 'se_resnet101' , 'new_splits'
    print (model_name,version,num_split)
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_train_tta',num_split),'rb')
    features=pickle.load(pickle_file)
    pickle_file.close()
    features.shape
    features=features.reshape(features.shape[0]//4,4,-1)
    features.shape
    
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_test',num_split),'rb')
    featurest=pickle.load(pickle_file)
    pickle_file.close()
    featurest.shape
    featurest=featurest.reshape(featurest.shape[0]//8,8,-1)
    featurest.shape
    features=torch.cat([features,featurest[:,:4]])
    features.shape
    dfm=pd.concat([old_train_df,old_test_df],sort=False,axis=0).reset_index(drop=True)[['PatientID']]
    atrain_df=dfm.merge(train_df,on='PatientID')
    split_train = atrain_df[atrain_df.PID.isin(set(split_sid[splits[num_split][0]]))].SeriesI.unique()
    split_validate =  atrain_df[atrain_df.PID.isin(set(split_sid[splits[num_split][1]]))].SeriesI.unique()
    print (split_train.shape,split_validate.shape)
    print (train_df[train_df.SeriesI.isin(set(split_train))].shape[0],
          train_df[train_df.SeriesI.isin(set(split_validate))].shape[0],)

    np.random.seed(SEED+num_split)
    torch.manual_seed(SEED+num_split)
    torch.cuda.manual_seed(SEED+num_split)
    torch.backends.cudnn.deterministic = True
    batch_size=16
    num_workers=12
    num_epochs=24
    klr=1
    weights = torch.tensor([1.,1.,1.,1.,1.,2.],device=device)
    train_dataset=FullHeadDataset(atrain_df,
                                  split_train,
                                  features,
                                  'SeriesI',
                                  'ImagePositionZ',
                                  hemorrhage_types,
                                  multi=multi)                
    validate_dataset=FullHeadDataset(atrain_df,
                                     split_validate,
                                     torch.cat([features[:,i,:] for i in range(4)],-1),
                                     'SeriesI',
                                     'ImagePositionZ',
                                     hemorrhage_types)                

    model=ResModelPool(features.shape[-1])
    version=version+'_fullhead_resmodel_pool2_stage2_{}'.format(multi)
    _=model.to(device)
    #mixup=Mixup(device=device)
    loss_func=my_loss
    #fig,ax = plt.subplots(figsize=(10,7))
    #gr=loss_graph(fig,ax,num_epochs,len(train_dataset)//batch_size+1,limits=[0.02,0.06])
    num_train_optimization_steps = num_epochs*(len(train_dataset)//batch_size+int(len(train_dataset)%batch_size>0))
    sched=WarmupExpCosineWithWarmupRestartsSchedule( t_total=num_train_optimization_steps, cycles=2,tau=1)
    optimizer = BertAdam(model.parameters(),lr=klr*1e-3,schedule=sched)
    history,best_model= model_train(model,
                                    optimizer,
                                    train_dataset,
                                    batch_size,
                                    num_epochs,
                                    loss_func,
                                    weights=weights,
                                    do_apex=False,
                                    validate_dataset=validate_dataset,
                                    param_schedualer=None,
                                    weights_data=None,
                                    metric=Metric(torch.tensor([1.,1.,1.,1.,1.,2.])),
                                    return_model=True,
                                    best_average=3,
                                    num_workers=num_workers,
                                    sampler=None,
                                    graph=None)
    torch.save(best_model.state_dict(), models_dir+models_format.format(model_name,version,num_split))

se_resnet101 new_splits 0


torch.Size([2697008, 256])

torch.Size([674252, 4, 256])

torch.Size([628360, 256])

torch.Size([78545, 8, 256])

...

{'loss': 0.061719958290806794, 'mloss': tensor(0.0647), 'val_loss': 0.07082761003339029, 'val_mloss_tot': tensor(0.0706)}


{'loss': 0.05716896897278506, 'mloss': tensor(0.0586), 'val_loss': 0.07069802660863478, 'val_mloss_tot': tensor(0.0705)}


{'loss': 0.05782607063883131, 'mloss': tensor(0.0566), 'val_loss': 0.07063272952241993, 'val_mloss_tot': tensor(0.0705)}


{'loss': 0.059804734933268974, 'mloss': tensor(0.0575), 'val_loss': 0.07075512491425862, 'val_mloss_tot': tensor(0.0706)}




0.07059992532563078


In [ ]:
%matplotlib nbagg
for num_split in range(5):
    multi=3
    model_name,version = 'se_resnet101' , 'new_splits_focal'
    print (model_name,version,num_split)
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_train_tta',num_split),'rb')
    features=pickle.load(pickle_file)
    pickle_file.close()
    features.shape
    features=features.reshape(features.shape[0]//4,4,-1)
    features.shape
    
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_test',num_split),'rb')
    featurest=pickle.load(pickle_file)
    pickle_file.close()
    featurest.shape
    featurest=featurest.reshape(featurest.shape[0]//8,8,-1)
    featurest.shape
    features=torch.cat([features,featurest[:,:4]])
    features.shape
    dfm=pd.concat([old_train_df,old_test_df],sort=False,axis=0).reset_index(drop=True)[['PatientID']]
    atrain_df=dfm.merge(train_df,on='PatientID')
    split_train = atrain_df[atrain_df.PID.isin(set(split_sid[splits[num_split][0]]))].SeriesI.unique()
    split_validate =  atrain_df[atrain_df.PID.isin(set(split_sid[splits[num_split][1]]))].SeriesI.unique()

    np.random.seed(SEED+num_split)
    torch.manual_seed(SEED+num_split)
    torch.cuda.manual_seed(SEED+num_split)
    torch.backends.cudnn.deterministic = True
    batch_size=16
    num_workers=12
    num_epochs=24
    klr=1
    weights = torch.tensor([1.,1.,1.,1.,1.,2.],device=device)
    train_dataset=FullHeadDataset(atrain_df,
                                  split_train,
                                  features,
                                  'SeriesI',
                                  'ImagePositionZ',
                                  hemorrhage_types,
                                  multi=multi)                
    validate_dataset=FullHeadDataset(atrain_df,
                                     split_validate,
                                     torch.cat([features[:,i,:] for i in range(4)],-1),
                                     'SeriesI',
                                     'ImagePositionZ',
                                     hemorrhage_types)                

    model=ResModelPool(features.shape[-1])
    version=version+'_fullhead_resmodel_pool2_stage2_{}'.format(multi)
    _=model.to(device)
    #mixup=Mixup(device=device)
    loss_func=my_loss
    #fig,ax = plt.subplots(figsize=(10,7))
    #gr=loss_graph(fig,ax,num_epochs,len(train_dataset)//batch_size+1,limits=[0.02,0.06])
    num_train_optimization_steps = num_epochs*(len(train_dataset)//batch_size+int(len(train_dataset)%batch_size>0))
    sched=WarmupExpCosineWithWarmupRestartsSchedule( t_total=num_train_optimization_steps, cycles=2,tau=1)
    optimizer = BertAdam(model.parameters(),lr=klr*1e-3,schedule=sched)
    history,best_model= model_train(model,
                                    optimizer,
                                    train_dataset,
                                    batch_size,
                                    num_epochs,
                                    loss_func,
                                    weights=weights,
                                    do_apex=False,
                                    validate_dataset=validate_dataset,
                                    param_schedualer=None,
                                    weights_data=None,
                                    metric=Metric(torch.tensor([1.,1.,1.,1.,1.,2.])),
                                    return_model=True,
                                    best_average=3,
                                    num_workers=num_workers,
                                    sampler=None,
                                    graph=None)
    torch.save(best_model.state_dict(), models_dir+models_format.format(model_name,version,num_split))

se_resnet101 new_splits_focal 0


torch.Size([2697008, 256])

torch.Size([674252, 4, 256])

torch.Size([628360, 256])

torch.Size([78545, 8, 256])

...

{'loss': 0.06044166597323735, 'mloss': tensor(0.0606), 'val_loss': 0.07132643982836287, 'val_mloss_tot': tensor(0.0712)}


{'loss': 0.05781309361027082, 'mloss': tensor(0.0578), 'val_loss': 0.07054092935985808, 'val_mloss_tot': tensor(0.0704)}


{'loss': 0.05795218775016159, 'mloss': tensor(0.0604), 'val_loss': 0.07080355528128016, 'val_mloss_tot': tensor(0.0706)}


{'loss': 0.057093642727518136, 'mloss': tensor(0.0558), 'val_loss': 0.07192068023695833, 'val_mloss_tot': tensor(0.0717)}


{'loss': 0.06261353620000616, 'mloss': tensor(0.0658), 'val_loss': 0.07041113815464817, 'val_mloss_tot': tensor(0.0702)}


In [23]:
%matplotlib nbagg
for num_split in range(5):
    multi=3
    model_name,version = 'se_resnext101_32x4d' , 'new_splits'
    print (model_name,version,num_split)
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_train_tta',num_split),'rb')
    features=pickle.load(pickle_file)
    pickle_file.close()
    features.shape
    features=features.reshape(features.shape[0]//4,4,-1)
    features.shape
    
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_test',num_split),'rb')
    featurest=pickle.load(pickle_file)
    pickle_file.close()
    featurest.shape
    featurest=featurest.reshape(featurest.shape[0]//8,8,-1)
    featurest.shape
    features=torch.cat([features,featurest[:,:4]])
    features.shape
    dfm=pd.concat([old_train_df,old_test_df],sort=False,axis=0).reset_index(drop=True)[['PatientID']]
    atrain_df=dfm.merge(train_df,on='PatientID')
    split_train = atrain_df[atrain_df.PID.isin(set(split_sid[splits[num_split][0]]))].SeriesI.unique()
    split_validate =  atrain_df[atrain_df.PID.isin(set(split_sid[splits[num_split][1]]))].SeriesI.unique()

    np.random.seed(SEED+num_split)
    torch.manual_seed(SEED+num_split)
    torch.cuda.manual_seed(SEED+num_split)
    torch.backends.cudnn.deterministic = True
    batch_size=16
    num_workers=12
    num_epochs=24
    klr=1
    weights = torch.tensor([1.,1.,1.,1.,1.,2.],device=device)
    train_dataset=FullHeadDataset(atrain_df,
                                  split_train,
                                  features,
                                  'SeriesI',
                                  'ImagePositionZ',
                                  hemorrhage_types,
                                  multi=multi)                
    validate_dataset=FullHeadDataset(atrain_df,
                                     split_validate,
                                     torch.cat([features[:,i,:] for i in range(4)],-1),
                                     'SeriesI',
                                     'ImagePositionZ',
                                     hemorrhage_types)                

    model=ResModelPool(features.shape[-1])
    version=version+'_fullhead_resmodel_pool2_stage2_{}'.format(multi)
    _=model.to(device)
    #mixup=Mixup(device=device)
    loss_func=my_loss
    #fig,ax = plt.subplots(figsize=(10,7))
    #gr=loss_graph(fig,ax,num_epochs,len(train_dataset)//batch_size+1,limits=[0.02,0.06])
    num_train_optimization_steps = num_epochs*(len(train_dataset)//batch_size+int(len(train_dataset)%batch_size>0))
    sched=WarmupExpCosineWithWarmupRestartsSchedule( t_total=num_train_optimization_steps, cycles=2,tau=1)
    optimizer = BertAdam(model.parameters(),lr=klr*1e-3,schedule=sched)
    history,best_model= model_train(model,
                                    optimizer,
                                    train_dataset,
                                    batch_size,
                                    num_epochs,
                                    loss_func,
                                    weights=weights,
                                    do_apex=False,
                                    validate_dataset=validate_dataset,
                                    param_schedualer=None,
                                    weights_data=None,
                                    metric=Metric(torch.tensor([1.,1.,1.,1.,1.,2.])),
                                    return_model=True,
                                    best_average=3,
                                    num_workers=num_workers,
                                    sampler=None,
                                    graph=None)
    torch.save(best_model.state_dict(), models_dir+models_format.format(model_name,version,num_split))

{'loss': 0.058057024460154504, 'mloss': tensor(0.0606), 'val_loss': 0.07031140612594675, 'val_mloss_tot': tensor(0.0698)}


{'loss': 0.05619916133718424, 'mloss': tensor(0.0558), 'val_loss': 0.07028087283430254, 'val_mloss_tot': tensor(0.0698)}


{'loss': 0.05533692743880959, 'mloss': tensor(0.0523), 'val_loss': 0.0699961722240423, 'val_mloss_tot': tensor(0.0695)}


{'loss': 0.05767238023349743, 'mloss': tensor(0.0575), 'val_loss': 0.0699991266188776, 'val_mloss_tot': tensor(0.0695)}


{'loss': 0.05897191727896761, 'mloss': tensor(0.0616), 'val_loss': 0.06991716339694758, 'val_mloss_tot': tensor(0.0695)}


...

{'loss': 0.06021073414059474, 'mloss': tensor(0.0630), 'val_loss': 0.07137734410881778, 'val_mloss_tot': tensor(0.0712)}


{'loss': 0.05607046722226902, 'mloss': tensor(0.0572), 'val_loss': 0.07121787283010099, 'val_mloss_tot': tensor(0.0710)}


{'loss': 0.05742730574831661, 'mloss': tensor(0.0563), 'val_loss': 0.0711681683285114, 'val_mloss_tot': tensor(0.0710)}


{'loss': 0.0588047465332592, 'mloss': tensor(0.0569), 'val_loss': 0.07118950749409723, 'val_mloss_tot': tensor(0.0710)}




0.07107733339480646


In [22]:
def align(arr,index1,index2):
    return arr[np.argsort(index2)[np.argsort(np.argsort(index1))]]

In [35]:
pred_list = []

In [23]:
pred_list = []
pred_list_tmp=[]
for num_split in tqdm_notebook(range(5)):
    model_name,version = 'se_resnext101_32x4d' , 'new_splits'
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_test',num_split),'rb')
    features=pickle.load(pickle_file)
    pickle_file.close()
    features=features.reshape(features.shape[0]//8,8,-1)

    model=ResModelPool(features.shape[-1])
    version=version+'_fullhead_resmodel_pool2_3'

    model.load_state_dict(torch.load(models_dir+models_format.format(model_name,version,num_split),map_location=torch.device(device)))
    test_dataset=train_dataset=FullHeadDataset(test_df,
                                  test_df.SeriesI.unique(),
                                  features,
                                  'SeriesI',
                                  'ImagePositionZ',multi=4)
    for i in tqdm_notebook(range(32),leave=False):
        pred_list.append(torch.sigmoid(model_run(model,test_dataset,do_apex=False,batch_size=128))[...,None])
        pred_list_tmp.append(pred_list[-1])
predse=torch.cat(pred_list_tmp,-1).mean(-1)

<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>

In [37]:

pred_list_tmp=[]
for num_split in tqdm_notebook(range(5)): #tqdm_notebook(range(5)):
    model_name,version = 'se_resnet101' , 'new_splits'
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_test',num_split),'rb')
    features=pickle.load(pickle_file)
    pickle_file.close()
    features=features.reshape(features.shape[0]//8,8,-1)

    model=ResModelPool(features.shape[-1])
    version=version+'_fullhead_resmodel_pool2_3'

    model.load_state_dict(torch.load(models_dir+models_format.format(model_name,version,num_split),map_location=torch.device(device)))
    test_dataset=train_dataset=FullHeadDataset(test_df,
                                  test_df.SeriesI.unique(),
                                  features,
                                  'SeriesI',
                                  'ImagePositionZ',multi=4)
    for i in tqdm_notebook(range(32),leave=False):
        pred_list.append(torch.sigmoid(model_run(model,test_dataset,do_apex=False,batch_size=128))[...,None])
        pred_list_tmp.append(pred_list[-1])
predres=torch.cat(pred_list_tmp,-1).mean(-1)
len(pred_list_tmp)

<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>

160

In [29]:
pred=torch.cat(pred_list,-1).mean(-1)

In [30]:
len(pred_list)

288

In [31]:
images_id_list=[]
dummeys=[]
image_arr=test_df.PatientID.values
ref_arr=test_df.SeriesI.values
order_arr=test_df.ImagePositionZ.values
for s in tqdm_notebook(test_df.SeriesI.unique()):
    dumm=np.zeros(60)
    head_idx = np.where(ref_arr==s)[0]
    sorted_head_idx=head_idx[np.argsort(order_arr[head_idx])]
    images_id_list.append(image_arr[sorted_head_idx])
    dumm[0:head_idx.shape[0]]=1
    dummeys.append(dumm)
image_ids=np.concatenate(images_id_list)
preds=pred.reshape(pred.shape[0]*pred.shape[1],6).numpy()[np.concatenate(dummeys)==1]

image_ids.shape

preds.shape

(78545,)

(78545, 6)

In [32]:
submission_df=get_submission_ids(image_ids,torch.tensor(preds),do_sigmoid=False)
submission_df.head(12)
submission_df.shape
sub_num=55
submission_df.to_csv('/media/hd/notebooks/data/RSNA/submissions/submission{}.csv'.format(sub_num),
                                                                  index=False, columns=['ID','Label'])


,ID,Label
0,ID_000012eaf_any,0.001291
1,ID_000012eaf_epidural,0.000125
2,ID_000012eaf_intraparenchymal,0.000145
3,ID_000012eaf_intraventricular,0.000027
4,ID_000012eaf_subarachnoid,0.000148
5,ID_000012eaf_subdural,0.001049
6,ID_0000ca2f6_any,0.001782
7,ID_0000ca2f6_epidural,0.000063
8,ID_0000ca2f6_intraparenchymal,0.000239
9,ID_0000ca2f6_intraventricular,0.000028


(471270, 2)